In [8]:
import torch
import numpy as np
# import seaborn as sns
import matplotlib.pyplot as plt
import random
# plt.style.use('seaborn')
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import os
import pandas as pd
pd.set_option('display.min_rows', 500)
import warnings
warnings.filterwarnings('ignore')

path='../../data/'
train = pd.read_csv(f"{path}/train_data.csv")
test = pd.read_csv(f"{path}/test_data.csv")
sub = pd.read_csv(f"{path}/sample_submission.csv")

# 피처엔지니어링 및 전처리
범주형 피처는 라벨인코딩까지 하고 csv 형태로 저장

In [32]:
import time
from datetime import datetime

day_dict = {'Tuesday': 0,
 'Thursday': 1,
 'Friday': 2,
 'Wednesday' : 3,
 'Monday': 4,
 'Saturday': 5,
 'Sunday': 6}

def convert_time(s):
    timestamp = time.mktime(
        datetime.strptime(s, "%Y-%m-%d %H:%M:%S").timetuple()
    )
    return int(timestamp)

def feature_engineering(df):
    df2 = df.copy()
    #유저별 시퀀스를 고려하기 위해 아래와 같이 정렬
    df2.sort_values(by=['userID','Timestamp'], inplace=True)
    
    df2['big_category'] = df2.testId.map(lambda x:x[2]).astype(int)
    df2['mid_category'] = df2.testId.map(lambda x: int(x[-3:]))
    df2['problem_num'] = df2.assessmentItemID.map(lambda x: int(x[-3:]))
    
    df2['Timestamp_start'] = pd.to_datetime(df['Timestamp'])
    df2['Timestamp_fin'] = df2.groupby('userID')['Timestamp_start'].shift(-1)
    df2['solvetime'] = df2.Timestamp_fin - df2.Timestamp_start
    df2['solvesec_600'] = df2.solvetime.map(lambda x : x.total_seconds()).shift(1).fillna(0)
    # df2['solvesec_cat'] = pd.to_datetime(df2.Timestamp).dt.day_name().map(day_dict)
    df2.loc[df2.solvesec_600>=600,'solvesec_600']=0
    df2.loc[df2.solvesec_600<0,'solvesec_600']=0
    
    correct_t = df2.groupby(['testId'])['answerCode'].agg(['mean', 'std'])
    correct_t.columns = ["test_mean", "test_std"]
    correct_k = df2.groupby(['KnowledgeTag'])['answerCode'].agg(['mean', 'std'])
    correct_k.columns = ["tag_mean", 'tag_std']
    correct_b = df2.groupby(['big_category'])['answerCode'].agg(['mean', 'std', 'sum'])
    correct_b.columns = ["big_mean", 'big_std', 'big_sum']
    correct_a = df2.groupby(['assessmentItemID'])['answerCode'].agg(['mean', 'std'])
    correct_a.columns = ["assess_mean", 'assess_std']
    correct_u = df2.groupby(['userID'])['answerCode'].agg(['mean', 'std', 'sum'])
    correct_u.columns = ["user_mean", 'user_std', 'user_sum']
    
    
    df2 = pd.merge(df2, correct_t, on=['testId'], how="left")
    df2 = pd.merge(df2, correct_k, on=['KnowledgeTag'], how="left")
    df2 = pd.merge(df2, correct_b, on=['big_category'], how="left")
    df2 = pd.merge(df2, correct_a, on=['assessmentItemID'], how="left")
    df2 = pd.merge(df2, correct_a, on=['assessmentItemID'], how="left")
    

    # df["Timestamp"] = df["Timestamp"].apply(convert_time)
    
    return df2

In [33]:
train_df = feature_engineering(train)
test_df = feature_engineering(test)

In [34]:
all = pd.concat([train, test]).copy().reset_index(drop=True)
all.sort_values(by=['userID','Timestamp'], inplace=True)
all_df = feature_engineering(all)

In [164]:
# train_df.to_csv('../dkt/asset/train_fe_df.csv')
# test_df.to_csv('../dkt/asset/test_fe_df.csv')
# all_df.to_csv('../dkt/asset/all_fe_df.csv')

In [35]:
cate_cols = ['testId','assessmentItemID','KnowledgeTag','big_category','mid_category','problem_num', 'month', 'dayname']
for col in cate_cols:
    exec(col + '2idx = {v:k for k,v in enumerate(all_df["' + col + '"].unique())}')
    exec('train_df["' + col + '"] = train_df["' + col + '"].map(' + col + '2idx)')

In [36]:
cate_cols = ['testId','assessmentItemID','KnowledgeTag','big_category','mid_category','problem_num', 'month', 'dayname']
for col in cate_cols:
    exec(col + '2idx = {v:k for k,v in enumerate(all_df["' + col + '"].unique())}')
    exec('test_df["' + col + '"] = test_df["' + col + '"].map(' + col + '2idx)')

In [37]:
train_df.to_csv('../dkt/asset/train_fe_df.csv')
test_df.to_csv('../dkt/asset/test_fe_df.csv')
all_df.to_csv('../dkt/asset/all_fe_df.csv')

In [38]:
!pwd``

/opt/ml/input/main_dir/FeatrueEngineering


In [153]:
import torch
a = torch.rand([64,20,1])
a.dtype

torch.float32

In [157]:
import torch
torch.__version__
torch.cuda.is_available()

'1.13.0+cu117'

True

In [150]:
import torch.nn as nn
tmp = nn.Linear(in_features=1, out_features=21)
tmp(a)

torch.Size([64, 20, 21])

# 'after_fe_train_test_bigcategory_fe.pkl' 으로 적용하기
FE된 df를 불러온 후, 범주형 피처는 라벨인코딩까지 하고 csv 형태로 저장

In [9]:
## 1. 데이터 로딩
data_dir = '/opt/ml/input/data' # 경로
after_fe_path = os.path.join(data_dir, 'after_fe_train_test_bigcategory_fe.pkl')
df = pd.read_pickle(after_fe_path)

In [15]:
train_df = df[df['kind'] == 'train']

In [16]:
test_df = df[df['kind'] == 'test']

In [18]:
all = pd.concat([train_df, test_df]).copy().reset_index(drop=True)
all.sort_values(by=['userID','Timestamp'], inplace=True)

In [30]:
pd.set_option('display.max_columns', None)

In [31]:
train_df.head(2)

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,kind,uidIdx,assIdx,testIdx,user_correct_answer,user_total_answer,user_acc,month,day,hour,dayname,big_category,problem_num,mid_category,test_mean,test_std,test_sum,tag_mean,tag_std,tag_sum,Timestamp2,solvetime,solvesec,solvesec_3600,time_category,solvesec_cumsum,solvecumsum_category,big_category_acc,big_category_std,big_category_cumconut,big_category_answer,big_category_answer_log1p,big_category_user_cum_acc,big_category_user_acc,big_category_user_std
0,0,0,0,1,2020-03-24 00:17:11,0,train,0,5354,975,NaN,0,NaN,0,0,0,0,0,0,0,0.952667,0.212422,1429,0.957333,0.202239,718,2020-03-24 00:17:11,0 days 00:00:00,0.0,0.0,0,0.0,0,0.711898,0.453371,0,0.0,NaN,0.0,0.791908,0.406531
1,0,1,0,1,2020-03-24 00:17:14,1,train,0,5355,975,1.0,1,1.0,0,0,0,0,0,1,0,0.952667,0.212422,1429,0.917067,0.275818,3439,2020-03-24 00:17:14,0 days 00:00:03,3.0,3.0,1,3.0,1,0.711898,0.453371,1,1.0,0.693147,1.0,0.791908,0.406531


In [21]:
train_df.columns

Index(['userID', 'assessmentItemID', 'testId', 'answerCode', 'Timestamp',
       'KnowledgeTag', 'kind', 'uidIdx', 'assIdx', 'testIdx',
       'user_correct_answer', 'user_total_answer', 'user_acc', 'month', 'day',
       'hour', 'dayname', 'big_category', 'problem_num', 'mid_category',
       'test_mean', 'test_std', 'test_sum', 'tag_mean', 'tag_std', 'tag_sum',
       'Timestamp2', 'solvetime', 'solvesec', 'solvesec_3600', 'time_category',
       'solvesec_cumsum', 'solvecumsum_category', 'big_category_acc',
       'big_category_std', 'big_category_cumconut', 'big_category_answer',
       'big_category_answer_log1p', 'big_category_user_cum_acc',
       'big_category_user_acc', 'big_category_user_std'],
      dtype='object')

In [23]:
cate_cols = ['testId','assessmentItemID','KnowledgeTag','big_category','mid_category',
             'problem_num', 'month','dayname','day', 'hour','time_category','solvecumsum_category']
num_cols = ['user_correct_answer', 'user_total_answer', 'user_acc', 'test_mean', 'test_std', 'test_sum',
            'tag_mean', 'tag_std', 'tag_sum', 'solvesec_3600', 'big_category_acc', 'big_category_std', 'big_category_cumconut',
            'big_category_answer','big_category_answer_log1p','big_category_user_cum_acc','big_category_user_acc','big_category_user_std']

In [25]:
for col in cate_cols:
    exec(col + '2idx = {v:k for k,v in enumerate(all["' + col + '"].unique())}')
    exec('train_df["' + col + '"] = train_df["' + col + '"].map(' + col + '2idx)')

In [26]:
for col in cate_cols:
    exec(col + '2idx = {v:k for k,v in enumerate(all["' + col + '"].unique())}')
    exec('test_df["' + col + '"] = test_df["' + col + '"].map(' + col + '2idx)')

In [28]:
train_df.to_csv('../dkt/asset/train_fe_df.csv')
test_df.to_csv('../dkt/asset/test_fe_df.csv')
all.to_csv('../dkt/asset/all_fe_df.csv')

In [ ]:
data_dir = '/opt/ml/input/data' # 경로
    write_path=f'{data_dir}/after_fe_train_test_cumsum.pkl'
    df.to_pickle(write_path,index=False)

# 여러모델 실행

In [71]:
import os
os.environ['MKL_THREADING_LAYER'] = 'GNU'

In [136]:
!python ../dkt/train.py --model lstm --n_epochs 500 --patience 30 --lr 0.001 --hidden_dim 128

wandb: Currently logged in as: nahyun. Use `wandb login --relogin` to force relogin
wandb: Currently logged in as: nahyun (recsys8). Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.13.5
wandb: Run data is saved locally in /opt/ml/input/main_dir/FeatrueEngineering/wandb/run-20221205_092604-29liu0ov
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run crisp-yogurt-461
wandb: ⭐️ View project at https://wandb.ai/recsys8/Sequential
wandb: 🚀 View run at https://wandb.ai/recsys8/Sequential/runs/29liu0ov
wandb: WARNING Calling wandb.run.save without any arguments is deprecated.Changes to attributes are automatically persisted.
Start Training: Epoch 1
Training steps: 0 Loss: 0.6893821954727173
Training steps: 50 Loss: 0.6667741537094116
TRAIN AUC : 0.6712833448961489 ACC : 0.5546075085324232
VALID AUC : 0.7294817512528998 ACC : 0.663681592039801

saving model ...
Start Training: Epoch 2
Training steps: 0 Loss: 0.5825918316841125
Training steps

In [1]:
!python ../dkt/train.py --model lstmattn --n_epochs 500 --patience 30 --lr 0.001 --hidden_dim 128

/opt/conda/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
wandb: Currently logged in as: nahyun. Use `wandb login --relogin` to force relogin
cpu
wandb: Currently logged in as: nahyun (recsys8). Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.13.5
wandb: Run data is saved locally in /opt/ml/input/main_dir/FeatrueEngineering/wandb/run-20221206_014526-3cfrgfgs
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run rosy-vortex-526
wandb: ⭐️ View project at https://wandb.ai/recsys8/Sequential
wandb: 🚀 View run at https://wandb.ai/recsys8/Sequential/runs/3cfrgfgs
wandb: WARNING Calling wandb.run.save without any arguments is deprecated.Changes to attributes are automatically persisted.
Start Training: Epoch 1
Traceback (most recent call last):
  File "..

In [2]:
!python ../dkt/train.py --model lqtransformer --n_epochs 500 --patience 30 --lr 0.001 --hidden_dim 128

/opt/conda/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
wandb: Currently logged in as: nahyun. Use `wandb login --relogin` to force relogin
cpu
wandb: Currently logged in as: nahyun (recsys8). Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.13.5
wandb: Run data is saved locally in /opt/ml/input/main_dir/FeatrueEngineering/wandb/run-20221206_014624-nbke7e9a
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run fine-puddle-528
wandb: ⭐️ View project at https://wandb.ai/recsys8/Sequential
wandb: 🚀 View run at https://wandb.ai/recsys8/Sequential/runs/nbke7e9a
wandb: WARNING Calling wandb.run.save without any arguments is deprecated.Changes to attributes are automatically persisted.
Start Training: Epoch 1
Traceback (most recent call last):
  File "..